In [43]:
# annoyingly this library needs an old (3.10 version of python to install)
# c:\Program Files\Python310\python.exe -m pip install requests
# c:\Program Files\Python310\python.exe -m pip install python-dotenv
# c:\Program Files\Python310\python.exe -m pip install random
# c:\Program Files\Python310\python.exe -m pip install web3
# c:\Program Files\Python310\python.exe -m pip install eip712-structs 
# c:\Program Files\Python310\python.exe -m pip install numpy 

import requests
import os
import time
# import numpy as np # don't need this
from dotenv import load_dotenv
from random import randint
from eip712_structs import EIP712Struct, Address, Uint, Boolean, make_domain
from web3 import Web3
from eth_account import Account


load_dotenv()
# variables in .env need to be = AEVOKEY, AEVOSECRET, WALLETADDRESS, SIGNINGKEY

# also import an instrument ids dictionary 


True

In [7]:
aevoKey = os.getenv("AEVOKEY")
aevoSecret = os.getenv("AEVOSECRET")
walletAddress = os.getenv("WALLETADDRESS")
signingKey = os.getenv("SIGNINGKEY")

In [8]:
instIDList = {
    '10000SATS-PERP': '13882' ,
    '1000BONK-PERP': '11312' ,
    '1000PEPE-PERP': '7020' ,
    'AEVO-PERP': '22007' ,
    'ALT-PERP': '17513' ,
    'APT-PERP': '11642' ,
    'AR-PERP': '20735' ,
    'ARB-PERP': '7349' ,
    'ATOM-PERP': '13987' ,
    'AVAX-PERP': '9060' ,  
    'AXL-PERP': '21084' ,
    'BANANA-PERP': '7623' ,
    'BEAM-PERP': '12478' ,
    'BEAMX-PERP': '12522' ,
    'BITCOIN-PERP': '8833' ,
    'BLAST-PERP': '12418' ,
    'BLUR-PERP': '10008' ,
    'BLZ-PERP': '8583' ,
    'BNB-PERP': '4042' ,
    'BTC-PERP': '3396' ,
    'CANTO-PERP': '8797' ,
    'CRV-PERP': '5757' ,
    'DOGE-PERP': '11969' ,
    'DYDX-PERP': '9358' ,
    'DYM-PERP': '15822' ,
    'ETH-PERP': '1' ,
    'FRIEND-PERP': '6815' ,
    'GLMR-PERP': '8552' ,
    'HIFI-PERP': '8139' ,
    'HPOS-PERP': '8802' ,
    'ILV-PERP': '12479' ,
    'INJ-PERP': '10009' ,
    'JITO-PERP': '13193' ,
    'JUP-PERP': '12938' ,
    'LDO-PERP': '11460' ,
    'LINK-PERP': '8494' ,
    'MANTA-PERP': '16207' ,
    'MATIC-PERP': '5125' ,
    'MEME-PERP': '10504' ,
    'MINA-PERP': '10280' ,
    'MKR-PERP': '8341' ,
    'MYRO-PERP': '21526' ,
    'NMR-PERP': '7480' ,
    'NTRN-PERP': '11849' ,
    'OP-PERP': '7348' ,
    'ORDI-PERP': '13194' ,
    'OX-PERP': '8543' ,
    'PARCL-PERP': '20289' ,
    'PENDLE-PERP': '21316' ,
    'PIXEL-PERP': '18892' ,
    'PORTAL-PERP': '20181' ,
    'PRIME-PERP': '9764' ,
    'PYTH-PERP': '11768' ,
    'RNDR-PERP': '13091' ,
    'SEI-PERP': '6104' ,
    'SHFL-PERP': '19417' ,
    'SOL-PERP': '5197' ,
    'STRAX-PERP': '9605' ,
    'STRK-PERP': '19174' ,
    'SUI-PERP': '17791' ,
    'SYN-PERP': '7530' ,
    'T-PERP': '9451' ,
    'TAO-PERP': '11070' ,
    'TIA-PERP': '8588' ,
    'TON-PERP': '20813' ,
    'TRB-PERP': '9924' ,
    'TRX-PERP': '12123' ,
    'UMA-PERP': '17792' ,
    'W-PERP': '22006' ,
    'WIF-PERP': '21317' ,
    'WLD-PERP': '5514' ,
    'XRP-PERP': '5196' ,
    'ZETA-PERP': '18342' }

In [83]:

baseURL = "https://api.aevo.xyz/"
publicHeaders = {"accept": "application/json"}
privateHeaders = {
    "accept": "application/json",
    "AEVO-KEY": aevoKey,
    "AEVO-SECRET": aevoSecret
    }
decimals = 10**6

def getAevo(urlSuffix, headers = publicHeaders):
    fullUrl = baseURL + urlSuffix
    response = requests.get(fullUrl, headers = headers)
    return(response.json())

def deleteAevo(urlSuffix, payld = None):
    thisHeader = privateHeaders
    if payld:
        thisHeader["content-type"] = "application/json"
    fullUrl = baseURL + urlSuffix
    # need to check if this works with payload = None which would happen with deleteOrderID
    response = requests.delete(fullUrl, json = payld, headers = thisHeader)
    return(response.json())

def postAevo(urlSuffix, payld):
    thisHeader = privateHeaders
    thisHeader["content-type"] = "application/json"
    print(thisHeader)
    fullUrl = baseURL + urlSuffix
    response = requests.post(fullUrl, json = payld, headers = thisHeader)
    print(response)
    return(response.json())

def getWtAvePrice(size, orderbook):
    sizeLeft = float(size)
    wtAvePx = 0
    for i in range(len(orderbook)):
        if sizeLeft <= 0:
            break
        sizeUsed = min(sizeLeft, float(orderbook[i][1]))
        wtAvePx += float(orderbook[i][0]) * sizeUsed
        sizeLeft -= sizeUsed
    return (wtAvePx / (size - sizeLeft)) # just in case the order is bigger than the market depth

def getBidOfferForSize(size, orderbook):
    if (size <= 0):
        size = 1
    print(orderbook)
    return (getWtAvePrice(size, orderbook["bids"]), getWtAvePrice(size, orderbook["asks"]))

class SigningOrder(EIP712Struct):
    maker = Address()
    isBuy = Boolean()
    limitPrice = Uint(256)
    amount = Uint(256)
    salt = Uint(256)
    instrument = Uint(256)
    timestamp = Uint(256)

def generateAndSignOrder(isBuy, instrument, limitprice, size):
    instrumentID = int(instIDList[instrument])
    limitPrice=int(limitprice * decimals),
    amount=int(size * decimals),
    salt=randint(0,10**6),
    timestamp=int(time.time())
    order_struct=SigningOrder(maker=walletAddress,
                      isBuy=isBuy,
                      limitPrice=limitPrice,
                      amount=amount,
                      salt=salt,
                      instrument=instrumentID,
                      timestamp=timestamp)
    domain = make_domain(name="Aevo Mainnet", version="1", chainId=1)
    signable_bytes = Web3.keccak(order_struct.signable_bytes(domain=domain))
    signature = Account._sign_hash(signable_bytes, signingKey).signature.hex()
    placingOrder = {"maker": walletAddress,
                      "is_buy": isBuy,
                      "limit_price": limitPrice,
                      "amount": amount,
                      "salt": salt,
                      "instrument": instrumentID,
                      "timestamp": timestamp,
                      "signature": signature}
    return (placingOrder)



In [82]:
# Public API calls
# getAevo("assets") # this gets a list of all the underlyings - no extra args
# getAevo("index?asset=" + ticker) # retrieve timestamp and price of ticker (e.g. "BNB")
# getAevo("markets") # this gets the details of all assets
# ticker = "ETH"
# getAevo("markets?asset=" + ticker + "&instrument_type=PERPETUAL") # this gets the details of one asset
# instrument = "ETH-PERP"
# getAevo("orderbook?instrument_name=" + instrument) # get orderbook (e.g. ETH-PERP)
# getBidOfferForSize(10, getAevo("orderbook?instrument_name=ETH-PERP")) # get the bid and offer in 10

# Private API calls
# getAevo("account", headers = privateHeaders) # gets account details
# getAevo("portfolio", headers = privateHeaders) # gets active portfolio details
# getAevo("orders", headers = privateHeaders) # gets all open orders

# deleteAevo("orders-all") # deletes all orders across all products
# payld = {"asset": "ETH", "instrument_type": "PERPETUAL"}
# deleteAevo("orders-all",payld) # deletes all orders across just the payld asset
# orderID = "0xdfaabxxxxxxx"
# deleteAevo("orders/"+ orderID) # deletes a specific order

instID = instIDList["ETH-PERP"]
payld = generateAndSignOrder(isBuy= False, instrument= "SOL-PERP", limitprice=195, size=0.2) # signs order instructions and names things correctly - yes it's strange - blame them not me
postAevo("orders", payld=payld) # posts an order with instructions from payld





AttributeError: 'str' object has no attribute 'text'

In [84]:
instID = instIDList["ETH-PERP"]
payld = generateAndSignOrder(isBuy= False, instrument= "SOL-PERP", limitprice=195, size=0.2) # signs order instructions and names things correctly - yes it's strange - blame them not me
postAevo("orders", payld=payld) # posts an order with instructions from payld


AttributeError: 'tuple' object has no attribute 'to_bytes'